<a href="https://colab.research.google.com/github/soobook/PyTorch-DL/blob/main/code/PT02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2회차: PyTorch 설치 및 기초 코딩

### CUDA(Compute Unified Device Architecture)
- NVIDIA에서 만든 병렬 연산용 프로그래밍 프레임워크
- GPU를 범용 컴퓨팅(GPGPU: General Purpose GPU) 작업에 활용

In [1]:
import torch

print(f'파이토치 버전: {torch.__version__}')
print("GPU" if torch.cuda.is_available() else "CPU")

파이토치 버전: 2.8.0+cu126
GPU


In [2]:
# torch 및 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
# 하이퍼파라미터 설정
batch_size = 64
learning_rate = 0.001
epochs = 5

In [4]:
# 데이터 전처리 및 로딩
# 이미지 픽셀 값을 [0, 255] 정수 값에서 [0.0, 1.0] 범위의 float 텐서로 변환
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 20.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 482kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.50MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.48MB/s]


In [5]:
# 데이터를 미니배치 단위로 묶고, 자동으로 반복 가능한 형태로 변환
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# train_loader 정보 출력
print("=== Train Loader Info ===")
print(f"총 배치 수 (len): {len(train_loader)}")    # 전체 배치 개수
print(f"총 샘플 수: {len(train_loader.dataset)}")  # 전체 데이터 수
print(f"총 배치 수 (직접계산): {len(train_loader.dataset)//batch_size + 1}")

# 첫 번째 배치 샘플 shape 확인
for images, labels in train_loader:
    print(f"이미지 shape: {images.shape}")         # 예: [batch_size, 1, 28, 28]
    print(f"레이블 shape: {labels.shape}")         # 예: [batch_size]
    print(f"데이터 타입: {images.dtype}, 라벨 dtype: {labels.dtype}")
    break

=== Train Loader Info ===
총 배치 수 (len): 938
총 샘플 수: 60000
총 배치 수 (직접계산): 938
이미지 shape: torch.Size([64, 1, 28, 28])
레이블 shape: torch.Size([64])
데이터 타입: torch.float32, 라벨 dtype: torch.int64


In [7]:
# test_loader 정보 출력
print("\n=== Test Loader Info ===")
print(f"총 배치 수 (len): {len(test_loader)}")
print(f"총 샘플 수: {len(test_loader.dataset)}")

for images, labels in test_loader:
    print(f"이미지 shape: {images.shape}")
    print(f"레이블 shape: {labels.shape}")
    print(f"데이터 타입: {images.dtype}, 라벨 dtype: {labels.dtype}")
    break


=== Test Loader Info ===
총 배치 수 (len): 157
총 샘플 수: 10000
이미지 shape: torch.Size([64, 1, 28, 28])
레이블 shape: torch.Size([64])
데이터 타입: torch.float32, 라벨 dtype: torch.int64


In [8]:
# PyTorch의 기본 신경망 모듈 nn.Module을 상속받아 DNN 클래스 정의
class DNN(nn.Module):

    # 클래스 초기화 메서드: 모델의 레이어들을 정의
    def __init__(self):
        # 부모 클래스(nn.Module)의 생성자를 호출해 초기화
        super(DNN, self).__init__()
        # 입력 데이터를 1차원으로 평탄화(28x28 → 784차원)
        self.flatten = nn.Flatten()
        # 여러 층을 순차적으로 쌓은 신경망 정의
        self.model = nn.Sequential(
            # 입력층(784) → 은닉층(128)
            nn.Linear(28*28, 128),
            # 활성화 함수 ReLU 적용
            nn.ReLU(),
            # 은닉층(128) → 은닉층(64)
            nn.Linear(128, 64),
            # 활성화 함수 ReLU 적용
            nn.ReLU(),
            # 은닉층(64) → 출력층(10 클래스)
            nn.Linear(64, 10)
        )

    # 순전파(forward) 과정 정의
    def forward(self, x):
        # 입력 데이터를 평탄화하여 벡터 형태로 변환
        x = self.flatten(x)
        # 정의한 모델 레이어들을 통과시켜 출력 반환
        return self.model(x)

In [10]:
# GPU(cuda)가 사용 가능하면 GPU에, 불가능하면 CPU에 연산을 수행하도록 장치를 설정함
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DNN 모델 객체를 생성하고, 앞서 설정한 장치(device)로 모델을 이동시킴
model = DNN().to(device)
# PyTorch 2.0의 torch.compile 기능을 활용해 모델을 컴파일하여 실행 속도를 최적화함
compiled_model = torch.compile(model)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(compiled_model.parameters(), lr=learning_rate)

In [11]:
# 학습 루프
for epoch in range(epochs): # 전체 에폭 반복
    compiled_model.train() # 학습 모드 설정
    total_loss = 0         # 에폭별 손실값
    correct = 0            # 에폭별 정답수
    for data, target in train_loader: # 배치 반복
        data, target = data.to(device), target.to(device) # 입력/라벨 GPU로 이동

        # 순전파 → 손실 → 역전파 → 옵티마이저 스텝
        optimizer.zero_grad() # 옵티마이저의 기울기 0으로 초기화
        output = compiled_model(data) # 모델로 출력 값 저장
        loss = criterion(output, target) # 손실함수로 손실값 저장
        loss.backward()   # 역전파로 기울기 수정
        optimizer.step()  # 옵티마이저로 파러미터인 기중치와 편향 수정

        total_loss += loss.item()
        # 모델의 예측값(output.argmax(1))과 실제 정답(target)을 비교하여 맞춘 개수를 누적
        correct += (output.argmax(1) == target).sum().item()

    # 정확도 계산
    accuracy = 100. * correct / len(train_loader.dataset)
    # 에폭별 결과 출력
    print(f"Epoch {epoch+1}, Loss: {total_loss:8.4f}, Accuracy: {accuracy:.2f}%")

W1013 08:36:09.954000 576 torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode


Epoch 1, Loss: 318.9353, Accuracy: 90.27%
Epoch 2, Loss: 129.8727, Accuracy: 95.88%
Epoch 3, Loss:  88.4028, Accuracy: 97.20%
Epoch 4, Loss:  68.0790, Accuracy: 97.76%
Epoch 5, Loss:  52.8591, Accuracy: 98.20%


In [12]:
# 테스트 모드로 평가
compiled_model.eval()
correct = 0
# 평가할 때는 기울기(gradient) 계산이 필요 없음
# 메모리 사용을 줄이고 속도 향상됨
# autograd를 끄는 것으로, 오직 순전파(forward)만 수행
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = compiled_model(data)
        # 모델의 예측값(output.argmax(1))과 실제 정답(target)을 비교하여 맞춘 개수를 누적
        correct += (output.argmax(1) == target).sum().item()

test_accuracy = 100. * correct / len(test_loader.dataset)
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 97.49%


### 종료